In [ ]:
#1. Data Structure (AST)

In [ ]:
class Node:
    def __init__(self, node_type, left=None, right=None, value=None):
        self.type = node_type  # 'operator' or 'operand'
        self.left = left  # Left child node
        self.right = right  # Right child node
        self.value = value  # Operand value, e.g., 'age > 30' or 'AND'

    def __repr__(self):
        if self.type == 'operator':
            return f"({self.left} {self.value} {self.right})"
        else:
            return f"{self.value}"


In [ ]:
#Data stroage using SQLITE

In [ ]:
CREATE TABLE rules (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    rule_string TEXT NOT NULL,
    ast TEXT
);


In [ ]:
#API Design

In [ ]:
import re

def create_rule(rule_string):
    tokens = re.split(r'(\(|\)|\s+)', rule_string)
    tokens = [token for token in tokens if token.strip() != '']
    
    def parse_expression(tokens):
        stack = []
        while tokens:
            token = tokens.pop(0)
            if token == '(':
                stack.append(parse_expression(tokens))
            elif token == ')':
                break
            else:
                stack.append(token)
        return build_ast(stack)

    def build_ast(tokens):
        if len(tokens) == 1:
            return Node('operand', value=tokens[0])
        if 'AND' in tokens or 'OR' in tokens:
            op = 'AND' if 'AND' in tokens else 'OR'
            i = tokens.index(op)
            return Node('operator', left=build_ast(tokens[:i]), right=build_ast(tokens[i+1:]), value=op)
        return None
    
    return parse_expression(tokens)


In [ ]:
def combine_rules(rule_strings, operator='AND'):
    asts = [create_rule(rule) for rule in rule_strings]
    if len(asts) == 1:
        return asts[0]
    
    root = asts[0]
    for ast in asts[1:]:
        root = Node('operator', left=root, right=ast, value=operator)
    return root


In [ ]:
#evaluate rule

In [ ]:
def evaluate_rule(ast, data):
    if ast.type == 'operator':
        if ast.value == 'AND':
            return evaluate_rule(ast.left, data) and evaluate_rule(ast.right, data)
        elif ast.value == 'OR':
            return evaluate_rule(ast.left, data) or evaluate_rule(ast.right, data)
    else:
        # Example: "age > 30" -> split and evaluate
        key, operator, value = re.split(r'([><=]+)', ast.value)
        key = key.strip()
        value = int(value.strip())
        if operator == '>':
            return data[key] > value
        elif operator == '<':
            return data[key] < value
        elif operator == '=':
            return data[key] == value
        return False


In [ ]:
#sample testcases

In [ ]:
rule1_string = "((age > 30 AND department = 'Sales') OR (age < 25 AND department = 'Marketing')) AND (salary > 50000 OR experience > 5)"
rule1_ast = create_rule(rule1_string)
print(rule1_ast)


In [ ]:
#combined rule

In [ ]:
rule2_string = "((age > 30 AND department = 'Marketing')) AND (salary > 20000 OR experience > 5)"
combined_ast = combine_rules([rule1_string, rule2_string], operator='AND')
print(combined_ast)


In [ ]:
#test evaluation

In [ ]:
data1 = {"age": 35, "department": "Sales", "salary": 60000, "experience": 3}
data2 = {"age": 24, "department": "Marketing", "salary": 25000, "experience": 2}

print(evaluate_rule(rule1_ast, data1))  # Should return True
print(evaluate_rule(rule1_ast, data2))  # Should return False


In [ ]:
#error handling

In [ ]:
def validate_rule_string(rule_string):
    # Basic checks for malformed rules (e.g., unmatched parentheses, invalid operators)
    pass

def validate_attributes(data):
    # Validate if attributes in data match a predefined catalog of attributes
    pass
